<a href="https://colab.research.google.com/github/AmeyaKI/multimodal-asr/blob/main/asr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive, files
drive.mount('/content/drive')
import os

In [ ]:
import torch
import numpy as np
from transformers import AutoProcessor, AutoModelForCTC
import librosa
import os

In [ ]:
# randomly selected audio
sample_path = "/content/drive/MyDrive/colab_notebooks/asr/sample.wav"

In [ ]:
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU name:", torch.cuda.get_device_name(0))

In [ ]:
MODEL = 'nvidia/parakeet-ctc-1.1b' # train custom model ?
device = 'cuda'

processor = AutoProcessor.from_pretrained(MODEL)
model = AutoModelForCTC.from_pretrained(MODEL, dtype='auto')

model.eval()

In [ ]:
waveform, sample_rate = librosa.load(
    sample_path,
    sr=None,
    mono=True
)

In [ ]:
# resample to 16,000 hz
TARGET_SAMPLE_RATE = 16000
if sample_rate != TARGET_SAMPLE_RATE:
    waveform = librosa.resample(
        waveform,
        orig_sr = sample_rate,
        target_sr = TARGET_SAMPLE_RATE
    )
# normalizing audio amp
max_val = np.max(np.abs(waveform))
if max_val > 0:
    waveform /= max_val

In [ ]:
# calc model inputs
model_inputs = processor(
    waveform,
    sampling_rate = TARGET_SAMPLE_RATE,
    return_tensors = 'pt'
)
model_inputs = model_inputs.to(
    device = model.device,
    dtype = model.dtype
)

In [ ]:
# Verifying
print(model_inputs.input_features.dtype)
print(next(model.parameters()).dtype)

In [ ]:
# obtain logits
with torch.no_grad():
    # Ensure input_features are in bfloat16 to match model's dtype
    model_inputs.input_features = model_inputs.input_features.to(torch.bfloat16)
    logits = model(**model_inputs).logits
    # (batch size, time steps, vocab size)

In [ ]:
# decode and trnaslate logits to predicted text
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)

text = transcription[0]
print(text)

In [ ]:
# Accuracy: Character Error Rate